<a href="https://colab.research.google.com/github/DenizY98/pydemo/blob/main/Fallstudie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importieren der benötigten Bibliotheken
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp
import pandas as pd
# Initialisieren des Solvers
solver = pywraplp.Solver.CreateSolver('SCIP')

In [2]:
! git clone https://github.com/AlexKressner/Industrielles_Management

fatal: destination path 'Industrielles_Management' already exists and is not an empty directory.


In [3]:
path = "Industrielles_Management/Daten/Fallstudie/"

In [4]:
#dataframes laden aus Daten der CSV-Dateien
nachfrage_df = pd.read_csv(f"{path}/FLINK_Nachfrage.csv", sep=";", index_col=0)
standorte_df = pd.read_csv(f"{path}/FLINK_Standorte.csv", sep=";")


###Ziel:
Maximierung der bedienten Nachfrage innerhalb der vorgegebenen Lieferyeit und des Budgets.

###Indexmengen:
$ Q = \{q \mid q = (i,j), i \in \{0, \ldots, 12\}, j \in \{0, \ldots, 12\}\} $
Menge aller Quadranten $ q $ als $( i , j )$ Tupel


$ S = \{s \mid s \in \{0, \ldots, 9\}\} $
Menge aller potentieller DarkStores $ s $

In [5]:
Q = [(i, j) for i in range(13) for j in range(13)]  # Menge aller Quadranten als (i, j) Paare
S = standorte_df['Potenzielle_Standorte'].to_list()  # Menge aller potentieller DarkStores

###Variablen:
$ x_s \in \{0,1\} \quad \forall s \in S $: Binärvariable, die angibt, ob am Standort $ s $ ein Dark Store eingerichtet wird $1$ oder nicht $0$.

$ y_{qs} \in \{0,1\} \quad \forall q \in Q, \forall s \in S $: Binärvariable, die angibt, ob der Standort $ s $ den Quadranten $ q (i,j) $ bedient $1$ oder nicht $0$.

In [6]:
x = {}
for s in S:
  x[s] = solver.BoolVar(f'x[{s}]')

In [7]:
y = {}
for s in S:
    for q in Q:
        y[q, s] = solver.BoolVar(f'y[{q},{s}]')

###Dictionaries:

In [8]:
#Standorte Dataframe
Lagerumschlagleistung = standorte_df.set_index('Potenzielle_Standorte')['Lagerumschlagleistung'].to_dict()
Errichtungskosten = standorte_df.set_index('Potenzielle_Standorte')['Errichtungskosten'].to_dict()
Koordinaten = standorte_df.set_index('Potenzielle_Standorte')[['i_Koordinate', 'j_Koordinate']].to_dict('index')

In [10]:
# Umstrukturierung der nachfrage_df DataFrame, um die Spalten 'i' und 'j' zu erhalten
nachfrage_df = nachfrage_df.reset_index().melt(id_vars='index', var_name='j', value_name='Nachfrage')
nachfrage_df.rename(columns={'index': 'i'}, inplace=True)
nachfrage_df['j'] = nachfrage_df['j'].astype(int)  # Konvertierung der Spalte 'j' in den Typ int
# Erstellung des nachfrage_dict Wörterbuchs
Nachfrage = nachfrage_df.set_index(['i', 'j']).to_dict('index')

###Parameter:

$ D_q $: Nachfrage im Quadranten $ q $ als Bestellungen pro Tag.

$ L_s $: Lagerumschlagleistung des Standorts $ s $ als Bestellungen pro Tag.

$ C_s $: Kosten für die Einrichtung eines Dark Stores am Standort $ s $.

$ B $: Gesamtbudget für die Einrichtung von Dark Stores.

$ T $: Maximale Lieferzeit $10 Minuten$.

$ A $: Maximale Anzahl von Quadranten, die ein Fahrradkurier bedienen kann $5 Quadranten$.

$ S_{\text{max}} $: Maximal belieferbare Quadranten je Darkstore.

In [17]:
D = {q: Nachfrage[q]['Nachfrage'] for q in Q}  # Nachfrage im Quadranten q als Bestellungen pro Tag
L = {s: Lagerumschlagleistung[s] for s in S}  # Lagerumschlagleistung des Standorts s als Bestellungen pro Tag
C = {s: Errichtungskosten[s] for s in S}  # Kosten für die Einrichtung eines Dark Stores am Standort s
B = 1000000  # Gesamtbudget für die Einrichtung von Dark Stores
T = 20  # Maximale Lieferzeit 10 Minuten
A = T/2  # Maximale Anzahl von Quadranten, die ein Fahrradkurier bedienen kann 5 Quadranten

###Restriktionen:
####Nachfragerestriktionen: $
\sum_{s \in S} y_{qs} \leq 1 \quad \forall q \in Q $
Jeder Quadrant wird von höchstens einem Dark Store bedient
####Budgetrestriktion: $
\sum_{s \in S} x_s \cdot C_s \leq B $
Ein Dark Store kann nur eingerichtet werden, wenn das Budget dies zulässt
####Lagerumschlagleistungsrestriktionen: $
\sum_{q \in Q} y_{qs} \cdot D_{q} \leq x_s \cdot L_s \quad \forall s \in S $
Die Nachfrage eines Quadranten kann nur bedient werden, wenn die Lagerumschlagleistung ausreicht und ein Dark Store vorhanden ist
####Reichweitenrestriktion für Fahrradkuriere: $
\text{Strecke}(s, q) > 5 \Rightarrow y_{qs} = 0 \quad \forall q \in Q, \forall s \in S $
Die Anzahl der von einem Fahrradkurier zurückgelegten Quadranten $Strecke$ darf nie größer $A$ sein, falls bestätigt, dass Quadrant $q$ von Standort $s$ beliefert wird.
<!--
####Liefergebietsrestriktionen: $
\sum_{q \in Q} y_{qs} \leq S_{\text{max}} \cdot x_s \quad \forall s \in S
$
Die Anzahl der von einem Dark Store bedienten Quadranten ist auf $ S_{\text{max}} $ beschränkt -->

In [18]:
# Nachfragerestriktionen
for q in Q:
    solver.Add(solver.Sum([y[(q, s)] for s in S]) <= 1)

# Budgetrestriktion
solver.Add(solver.Sum([x[s] * C[s] for s in S]) <= B)

# Lagerumschlagleistungsrestriktionen
for s in S:
    solver.Add(solver.Sum([y[(q, s)] * D[q] for q in Q]) <= x[s] * L[s])

# Reichweitenrestriktion für Fahrradkuriere
for s in S:
    for q in Q:
        # Berechnung der Manhattan-Distanz zwischen dem Standort s und dem Quadranten q
        distance = abs(Koordinaten[s]['i_Koordinate'] - q[0]) + abs(Koordinaten[s]['j_Koordinate'] - q[1])
        if distance > 5:
            solver.Add(y[q, s] == 0)
# Liefergebietsrestriktionen
# Smax = 1  # Maximale Anzahl von Quadranten, die ein Standort bedienen kann
# for s in S:
#     solver.Add(solver.Sum([y[(q, s)] for q in Q]) <= Smax * x[s])


###Zielfunktion:
Maximiere die gesamte bediente Nachfrage:

$ \text{Max} \sum_{sq} y_{qs} \cdot D_q $

In [19]:
# Zielfunktion
solver.Maximize(solver.Sum([y[q, s] * D[q] for q in Q for s in S]))
# Lösen des Modells
status = solver.Solve()

In [20]:
# Ausgabe der Lösung
if status == pywraplp.Solver.OPTIMAL:
    print('Lösung gefunden:')
    for s in S:
        if x[s].solution_value() == 1:
            print(f'Standort {s} wird eingerichtet.')
            for q in Q:
                if y[q, s].solution_value() == 1:
                    print(f'Quadrant {q} wird von Standort {s} bedient.')
else:
    print('Keine optimale Lösung gefunden.')

Lösung gefunden:
Standort 1 wird eingerichtet.
Quadrant (6, 12) wird von Standort 1 bedient.
Quadrant (7, 11) wird von Standort 1 bedient.
Quadrant (7, 12) wird von Standort 1 bedient.
Quadrant (8, 12) wird von Standort 1 bedient.
Quadrant (9, 8) wird von Standort 1 bedient.
Quadrant (9, 10) wird von Standort 1 bedient.
Quadrant (9, 11) wird von Standort 1 bedient.
Quadrant (9, 12) wird von Standort 1 bedient.
Quadrant (10, 10) wird von Standort 1 bedient.
Quadrant (11, 11) wird von Standort 1 bedient.
Quadrant (12, 10) wird von Standort 1 bedient.
Standort 2 wird eingerichtet.
Quadrant (8, 1) wird von Standort 2 bedient.
Quadrant (9, 0) wird von Standort 2 bedient.
Quadrant (9, 1) wird von Standort 2 bedient.
Quadrant (9, 2) wird von Standort 2 bedient.
Quadrant (9, 3) wird von Standort 2 bedient.
Quadrant (10, 0) wird von Standort 2 bedient.
Quadrant (10, 1) wird von Standort 2 bedient.
Quadrant (10, 2) wird von Standort 2 bedient.
Quadrant (11, 0) wird von Standort 2 bedient.
Quadra

In [21]:
# #Backup---
# # Indexmengen
# I = [(i, j) for i in range(13) for j in range(13)]  # Menge aller Quadranten als (i, j) Paare
# J = standorte_df['Potenzielle_Standorte'].to_list()  # Menge aller potentieller DarkStores

# # Variablen
# x = {}  # x[(i, j)][k] ist eine Binärvariable, die angibt, ob Standort k Quadrant (i, j) bedient
# y = {}  # y[k] ist eine Binärvariable, die angibt, ob ein Dark Store am Standort k eingerichtet wird

# # Hinzufügen der Variablen zum Solver
# for (i, j) in I:
#     for k in J:
#         x[(i, j), k] = solver.BoolVar(f'x[{(i, j)},{k}]')

# for k in J:
#     y[k] = solver.BoolVar(f'y[{k}]')

# # Parameter
# nachfrage_dict = nachfrage_df.set_index(['i', 'j']).to_dict('index')  # Umwandlung der Nachfrage DataFrame in ein Wörterbuch
# D = { (i, j): nachfrage_dict[(i, j)][str(j)] for (i, j) in I }  # Nachfrage im Quadranten (i, j) (Bestellungen pro Tag)
# L = standorte_df.set_index('Potenzielle_Standorte')['Lagerumschlagleistung'].to_dict()  # Lagerumschlagleistung des Standorts k (Bestellungen pro Tag)
# C = standorte_df.set_index('Potenzielle_Standorte')['Errichtungskosten'].to_dict()  # Kosten für die Einrichtung eines Dark Stores am Standort k

# # Restliche Parameter und Restriktionen bleiben unverändert
# B = 1000000  # Gesamtbudget für die Einrichtung von Dark Stores
# S = 5  # Maximale Anzahl von Quadranten, die ein Fahrradkurier bedienen kann
# # Zielfunktion
# solver.Maximize(solver.Sum([x[(i, j), k] * D[(i, j)] for (i, j) in I for k in J]))

# # Indexmengen
# I = [(i, j) for i in range(13) for j in range(13)]  # Menge aller Quadranten als (i, j) Paare
# J = standorte_df['Potenzielle_Standorte']  # Menge aller potentieller DarkStores

# # Variablen
# x = {}  # x[(i, j)][k] ist eine Binärvariable, die angibt, ob Standort k Quadrant (i, j) bedient
# y = {}  # y[k] ist eine Binärvariable, die angibt, ob ein Dark Store am Standort k eingerichtet wird

# # Hinzufügen der Variablen zum Solver
# for (i, j) in I:
#     for k in J:
#         x[(i, j), k] = solver.BoolVar(f'x[{(i, j)},{k}]')

# for k in J:
#     y[k] = solver.BoolVar(f'y[{k}]')

# # Parameter
# D = { (i, j): nachfrage_df.loc[i, str(j)] for (i, j) in I }  # Nachfrage im Quadranten (i, j) (Bestellungen pro Tag)
# L = { k: standorte_df.loc[k, 'Lagerumschlagleistung'] for k in J }  # Lagerumschlagleistung des Standorts k (Bestellungen pro Tag)
# C = { k: standorte_df.loc[k, 'Errichtungskosten'] for k in J }  # Kosten für die Einrichtung eines Dark Stores am Standort k
# B = 1000000  # Gesamtbudget für die Einrichtung von Dark Stores
# S = 5  # Maximale Anzahl von Quadranten, die ein Fahrradkurier bedienen kann

# # Restriktionen
# # Budgetrestriktion
# solver.Add(solver.Sum([y[k] * C[k] for k in J]) <= B)

# # Nachfragerestriktionen
# for (i, j) in I:
#     solver.Add(solver.Sum([x[(i, j), k] for k in J]) <= 1)

# # Lagerumschlagleistungsrestriktionen
# for k in J:
#     solver.Add(solver.Sum([x[(i, j), k] * D[(i, j)] for (i, j) in I]) <= y[k] * L[k])

# # Liefergebietsrestriktionen
# for k in J:
#     solver.Add(solver.Sum([x[(i, j), k] for (i, j) in I]) <= S * y[k])

# # Reichweitenrestriktion für Fahrradkuriere
# for k in J:
#     for (i, j) in I:
#         if y[k].solution_value() == 1:
#             # Berechnung der Entfernung zwischen dem Standort k und dem Quadranten (i, j)
#             distance = abs(standorte_df.loc[k, 'i_Koordinate'] - i) + abs(standorte_df.loc[k, 'j_Koordinate'] - j)
#             # Hinzufügen der Reichweitenrestriktion
#             solver.Add(x[(i, j), k] == 0 if distance > 5 else x[(i, j), k] >= 0)

# # Zielfunktion
# solver.Maximize(solver.Sum([x[(i, j), k] * D[(i, j)] for (i, j) in I for k in J]))

# # Lösen des Modells
# status = solver.Solve()

# # Ausgabe der Lösung
# if status == pywraplp.Solver.OPTIMAL:
#     print('Lösung gefunden:')
#     for k in J:
#         if y[k].solution_value() == 1:
#             print(f'Standort {k} wird eingerichtet.')
#             for (i, j) in I:
#                 if x[(i, j), k].solution_value() == 1:
#                     print(f'Quadrant {(i, j)} wird von Standort {k} bedient.')
# else:
#     print('Keine optimale Lösung gefunden.')